In [1]:
from openfisca_survey_manager.scenarios import AbstractSurveyScenario
from openfisca_france import CountryTaxBenefitSystem
from openfisca_france.model.base import Famille, FoyerFiscal, Menage
import pandas as pd


In [2]:
class StrasbourgSurveyScenario(AbstractSurveyScenario):
    def __init__(self, data = None):
        super(StrasbourgSurveyScenario, self).__init__()

        tax_benefit_system = CountryTaxBenefitSystem()
        tax_benefit_system.load_extension('openfisca_france_local')

        input_data_frame = data.get('input_data_frame')
        self.used_as_input_variables = list(
            set(tax_benefit_system.variables.keys()).intersection(
                set(input_data_frame.columns)
                ))

        self.set_tax_benefit_systems(tax_benefit_system)
        self.year = "2021-03"
        self.init_from_data(data = data)


In [3]:
qf = [410, 510, 620, 720, 820, 920, 1030, 1540, 2050, 2051, 2052]

In [4]:
counts = [4025, 2113, 1832, 1603, 962, 638, 566, 1990, 1157, 2090, 45]

In [38]:
input_data_frame = pd.DataFrame({
    'strasbourg_metropole_quotient_familial': qf,
    'famille_role_index': 0,
    'foyer_fiscal_role_index': 0,
    'menage_role_index': 0,
    'famille_id': range(len(qf)),
    'foyer_fiscal_id': range(len(qf)),
    'menage_id': range(len(qf)),
    })
input_data_frame

,strasbourg_metropole_quotient_familial,famille_role_index,foyer_fiscal_role_index,menage_role_index,famille_id,foyer_fiscal_id,menage_id
0,410,0,0,0,0,0,0
1,510,0,0,0,1,1,1
2,620,0,0,0,2,2,2
3,720,0,0,0,3,3,3
4,820,0,0,0,4,4,4
5,920,0,0,0,5,5,5
6,1030,0,0,0,6,6,6
7,1540,0,0,0,7,7,7
8,2050,0,0,0,8,8,8
9,2051,0,0,0,9,9,9


In [39]:
data = dict(input_data_frame = input_data_frame)
scenario = StrasbourgSurveyScenario(data = data)

# Tarif des cantines par familles

In [40]:
tarif_cantine = scenario.simulation.calculate('strasbourg_metropole_tarification_cantine', period = '2021-03')
tarif_cantine

array([1.5 , 2.15, 2.15, 2.15, 2.15, 2.15, 2.15, 2.15, 2.15, 2.15, 2.15],
      dtype=float32)

# Recettes par tranches / types de familles

In [44]:
sum_tranches = counts * tarif_cantine * 200 # Pour 200 repas par an
sum_tranches

array([1207500.        ,  908590.04030228,  787760.03494263,
        689290.0305748 ,  413660.01834869,  274340.01216888,
        243380.01079559,  855700.03795624,  497510.02206802,
        898700.03986359,   19350.00085831])

# Recettes totales

In [45]:
sum(sum_tranches)

6795780.247879028